Because there is so much Green taxi trip data (16.4 million rows), I wanted to dedicate a job to reading in the data, doing some basic filtering, and saving off two months (April and March).  This will make it easier and less time consuming to clean up the data in the future since I can read files that are already somewhat cleaned and can avoid running all of this code over and over as I figure out what features are important.

In [1]:
import pandas as pd
from datetime import datetime
from geopy.distance import vincenty

pd.set_option('display.max_columns', None)

In [2]:
green_taxi = pd.read_csv("2016_Green_Taxi_Trip_Data.csv")

In [3]:
green_taxi.count()

VendorID                 16385532
lpep_pickup_datetime     16385532
Lpep_dropoff_datetime    16385532
Store_and_fwd_flag       16385532
RateCodeID               16385532
Pickup_longitude          9018030
Pickup_latitude           9018030
Dropoff_longitude         9018030
Dropoff_latitude          9018030
Passenger_count          16385532
Trip_distance            16385532
Fare_amount              16385532
Extra                    16385532
MTA_tax                  16385532
Tip_amount               16385532
Tolls_amount             16385532
Ehail_fee                       0
improvement_surcharge    16385532
Total_amount             16385532
Payment_type             16385532
Trip_type                16385060
PULocationID              7367502
DOLocationID              7367502
dtype: int64

In [4]:
green_taxi_with_geo = green_taxi[green_taxi["Pickup_longitude"].notnull()]

In [5]:
green_taxi_with_geo[0:5]

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,improvement_surcharge,Total_amount,Payment_type,Trip_type,PULocationID,DOLocationID
0,2,01/01/2016 12:29:24 AM,01/01/2016 12:39:36 AM,N,1,-73.928642,40.680611,-73.924278,40.698044,1,1.46,8.0,0.5,0.5,1.86,0.0,NaN,0.3,11.16,1,1.0,NaN,NaN
1,2,01/01/2016 12:19:39 AM,01/01/2016 12:39:18 AM,N,1,-73.952675,40.723175,-73.923920,40.761379,1,3.56,15.5,0.5,0.5,0.00,0.0,NaN,0.3,16.80,2,1.0,NaN,NaN
2,2,01/01/2016 12:19:33 AM,01/01/2016 12:39:48 AM,N,1,-73.971611,40.676105,-74.013161,40.646072,1,3.79,16.5,0.5,0.5,4.45,0.0,NaN,0.3,22.25,1,1.0,NaN,NaN
3,2,01/01/2016 12:22:12 AM,01/01/2016 12:38:32 AM,N,1,-73.989502,40.669579,-74.000648,40.689034,1,3.01,13.5,0.5,0.5,0.00,0.0,NaN,0.3,14.80,2,1.0,NaN,NaN
4,2,01/01/2016 12:24:01 AM,01/01/2016 12:39:22 AM,N,1,-73.964729,40.682854,-73.940720,40.663013,1,2.55,12.0,0.5,0.5,0.00,0.0,NaN,0.3,13.30,2,1.0,NaN,NaN


I will not be using any of the below columns as features, or as a way of filtering the data, so I'm just going to remove this data first.

The initial charge on a taxi fare is $2.50 http://www.nyc.gov/html/tlc/html/passenger/taxicab_rate.shtml.  Will limit to 2.0 in the case of negotiated fare.

In addition, the base rate for a mile is \$2.50.  Since it looks like a driver may negotiate a fare, we are going to assume the driver wouldn't go any lower than 50 cents a mile.  In addition, very, very, very (very, very) few people would pay over \$5000/mile (note this is equivalent to \$50 for 0.01 mile - about a block) in base rate.  Both of these point to some piece of data being off.

I am also going to only keep records with rate codes that are a 1 (standard fare) or a 5 (negotiated fares) as these account for most of the data.

In [6]:
green_taxi_with_geo = green_taxi_with_geo[(green_taxi_with_geo["Trip_distance"] > 0) & (green_taxi_with_geo["Fare_amount"] >= 2)]
green_taxi_with_geo = green_taxi_with_geo[((green_taxi_with_geo["Fare_amount"]/green_taxi_with_geo["Trip_distance"]) >= 0.5) & ((green_taxi_with_geo["Fare_amount"]/green_taxi_with_geo["Trip_distance"]) <= 5000)]
green_taxi_with_geo = green_taxi_with_geo[(green_taxi_with_geo["RateCodeID"] == 1) | (green_taxi_with_geo["RateCodeID"] == 5)]

In [7]:
green_taxi_with_geo = green_taxi_with_geo.drop(labels = [
    "PULocationID", "DOLocationID", "Ehail_fee", 
    "Fare_amount", "Extra", "MTA_tax", "Tip_amount", 
    "Tolls_amount", "Total_amount", 
    "Payment_type", "improvement_surcharge"], axis = 1)

In [8]:
def getDate(d, format):
    if not isinstance(d, str):
        return None
    return datetime.strptime(d, format)

green_taxi_with_geo["lpep_pickup_datetime"] = green_taxi_with_geo["lpep_pickup_datetime"].map(lambda x: getDate(x, "%m/%d/%Y %I:%M:%S %p"))
green_taxi_with_geo["Lpep_dropoff_datetime"] = green_taxi_with_geo["Lpep_dropoff_datetime"].map(lambda x: getDate(x, "%m/%d/%Y %I:%M:%S %p"))

New York City geo points are around (40, -73).  Because of this, I am going to do an initial filter to remove records where any of the geopoints have a 0 as the latitude or longitude.  I am also going to filter the geo down to points that are specifically within a polygon that's loosely around New York City; however to save time, I will perform that calculation only on the train/validation/test data in the next notebook. 

In addition, I noticed there were a few duplicate records, so I'm going to drop those.  The duplicate records could be due to groups travelling together, but I don't see much importance in keeping that information around; one record is good enough for my purposes.

In [9]:
green_taxi_with_geo = green_taxi_with_geo[(green_taxi_with_geo["Pickup_longitude"] != 0) & 
                                    (green_taxi_with_geo["Pickup_latitude"] != 0) &
                                    (green_taxi_with_geo["Dropoff_longitude"] != 0) & 
                                    (green_taxi_with_geo["Dropoff_latitude"] != 0)]


green_taxi_with_geo = green_taxi_with_geo.drop_duplicates()

In [10]:
green_taxi_with_geo.to_pickle("data/greenTaxiWithGeo")

In [11]:
# green_taxi_with_geo = pd.read_pickle("data/greenTaxiWithGeo")

In [12]:
import geojson
import shapely
from shapely import geometry

nyc_zone_geojson_string = open("data/zone.geojson").read()
nyc_zone_geojson = geojson.loads(nyc_zone_geojson_string)

nyc_polygon = shapely.geometry.polygon.Polygon([(pt[1], pt[0]) for pt in nyc_zone_geojson["features"][0]["geometry"]["coordinates"][0]])

def checkIfPointIsInNYC(p_lat, p_long, d_lat, d_long):
    return (nyc_polygon.contains(shapely.geometry.Point(p_lat, p_long))) & (nyc_polygon.contains(shapely.geometry.Point(d_lat, d_long)))

def calculateDistance(p_lat, p_long, d_lat, d_long):
    return vincenty((p_lat, p_long), (d_lat, d_long)).miles

I decided to use April data as train/validate/test data.  In particular, I am going to train/validate on the first 3ish weeks of April and test using the last week of April.  This is to limit the number of data points to train over since all of April is still about 1.4 million records.  In addition, I thought it best to train over a continuous time segment as well as test over a continuous time segment because there is some correlation between the taxi rides minute to minute because taxis can't just transport themselves.  I am also going to see how well the final model will run over March data to see how robust the model is against different information.  I'm not expecting the model to transfer to March data well, but I'm curious and thought it would be an interesting thing to explore for this project.

First, I filter the data for data that has pickup times in April.  Then, I filter that data only to Pickup/Dropoff geopoints that are within a loose polygon around NYC to ensure that I'm looking at records that are relevant to what we want to train on.  In addition, if the start and end point are outside of NYC, in most cases the record is going to be "bad" data. Lastly, I calculate the Vincenty distance between the pickup and dropoff geopoints; this is the "as the bird flies" distance between the two geopoints, so it is the minimum distance that the odometer should read for the trip.  I do allow for the odometer trip distance calculation to have up to less than 0.1 miles of the Vincenty calculation to account for potential calculation discrepancies of the Vincenty calculation as well as minor discrepancies in the start/stop geos.

In [13]:
april_data = green_taxi_with_geo[green_taxi_with_geo["lpep_pickup_datetime"].map(lambda x: x.month) == 4]

april_data = april_data[april_data[["Pickup_latitude", "Pickup_longitude", "Dropoff_latitude", "Dropoff_longitude"]].apply(lambda x: checkIfPointIsInNYC(*x), axis=1)]

april_data["VincentyMiles"] = april_data[["Pickup_latitude", "Pickup_longitude", "Dropoff_latitude", "Dropoff_longitude"]].apply(lambda x: calculateDistance(*x), axis=1)
april_data = april_data[(april_data["Trip_distance"] - april_data["VincentyMiles"]) >= -0.1]
april_data = april_data.drop(labels = ["Trip_distance"], axis = 1)

In [14]:
april_data.to_pickle("data/allAprilData")

Now, I perform the same filtering on the March data.

In [15]:
march_data = green_taxi_with_geo[green_taxi_with_geo["lpep_pickup_datetime"].map(lambda x: x.month) == 3]

march_data = march_data[march_data[["Pickup_latitude", "Pickup_longitude", "Dropoff_latitude", "Dropoff_longitude"]].apply(lambda x: checkIfPointIsInNYC(*x), axis=1)]

march_data["VincentyMiles"] = march_data[["Pickup_latitude", "Pickup_longitude", "Dropoff_latitude", "Dropoff_longitude"]].apply(lambda x: calculateDistance(*x), axis=1)
march_data = march_data[(march_data["Trip_distance"] - march_data["VincentyMiles"]) >= -0.1]
march_data = march_data.drop(labels = ["Trip_distance"], axis = 1)

In [16]:
march_data.to_pickle("data/allMarchData")